# La organización Kiva de préstamos entre particulares

### Disponible en Kaggle en:
https://www.kaggle.com/kiva/data-science-for-good-kiva-crowdfunding/version/5

El conjunto de datos elegido recoge estadísticas de Kiva entre 2014 y 2017. Kiva abre un nuevo mundo de oportunidades para los menos favorecidos y nos permite a cualquiera de nosotros convertirnos en superhéroes. Se trata de una organización sin ánimo de lucro que ofrece pequeños préstamos para ayudar a las comunidades desatendidas que no tienen acceso a los servicios bancarios normales. Proporciona una plataforma y une a personas que estén dispuestas a prestar, un mínimo de 25 dólares, y prestatarios que expongan sus necesidades, la finalidad del dinero y las condiciones de devolución,  porque no es una donación, es un préstamo.

Al crear este servicio, Kiva habilita una solución que desbloquea capital para todos y mantiene un interés financiero muy bajo para los prestatarios. Por otro lado, permite que cualquiera sea parte de la solución y brinda a las personas un amplio abanico de opciones para elegir quién, dónde, cuánto y para qué sector desea ayudar.

### Variables y significado

Las variables utilizadas para describir cada préstamo son:


* id: ID único para préstamo
* funding_amount: la cantidad desembolsada por Kiva al agente de campo (USD)
* loan_amount: la cantidad desembolsada por el agente de campo al prestatario (USD)
* country_code: código ISO del país en el que se desembolsó el préstamo
* activity: Categoría más granular
* sector: Categoría de alto nivel
* use: Uso exacto del monto del préstamo
* country: Nombre completo del país en el que se desembolsó el préstamo
* region: Nombre completo de la región dentro del país
* currency: La moneda en que se desembolsó el préstamo
* partner_id: ID de la organización asociada
* posted_time: Hora a la que el agente de campo (intermediario) publica el préstamo en Kiva
* disbursed_time: Hora en que el agente de campo (intermediario) entrega el préstamo al beneficiario
* funded_time: El momento en que el préstamo publicado en Kiva es financiado por los prestamistas por completo
* term_in_months: La duración por la cual el préstamo se desembolsó en meses
* lender_count: El número total de prestamistas que contribuyeron a este préstamo.
* tags: Etiquetas para describir el caso específico
* borrower_genders: letras M, F separadas por comas, donde cada instancia representa un solo hombre / mujer en el grupo
* repayment_interval: Estado de pago
* date: fecha en la base de datos de esta operación.

**Nombre completo del alumno:**  NAIRA CARRUCCIO VILLADA

# INSTRUCCIONES

En cada celda debes responder a la pregunta formulada, asegurándote de que el resultado queda guardado en la(s) variable(s) que por defecto vienen inicializadas a `None`. No se necesita usar variables intermedias, pero puedes hacerlo siempre que el resultado final del cálculo quede guardado exactamente en la variable que venía inicializada a None (debes reemplazar None por la secuencia de transformaciones necesarias, pero nunca cambiar el nombre de esa variable). 

**No olvides borrar la línea *raise NotImplementedError()* de cada celda cuando hayas completado la solución de esa celda y quieras probarla**.

Después de cada celda evaluable verás una celda con código. Ejecútala (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no. Asegúrate de que, al menos, todas las celdas indican que el código es correcto antes de enviar el notebook terminado.

**Nunca se debe redondear ninguna cantidad si no lo pide explícitamente el enunciado**

### Cada solución debe escribirse obligatoriamente en la celda habilitada para ello. Cualquier celda adicional que se haya creado durante el desarrollo deberá ser eliminada.

Si necesitas crear celdas auxiliares durante el desarrollo, puedes hacerlo pero debes asegurarte de borrarlas antes de entregar el notebook.

### Sobre el dataset kiva_loans.csv se pide:

**(1 punto)** Ejercicio 1

* Leerlo **sin intentar** que Spark infiera el tipo de dato de cada columna
* Puesto que existen columnas que contienen una coma enmedio del valor, en esos casos los valores vienen entre comillas dobles. Spark ya contempla esta posibilidad y puede leerlas adecuadamente **si al leer le indicamos las siguientes opciones adicionales** además de las que ya sueles usar: `.option("quote", "\"").option("escape", "\"")`.
* Asegúrate de que las **filas que no tienen el formato correcto sean descartadas**, indicando también la opción `mode` con el valor `DROPMALFORMED` como vimos en clase.
* Crear un nuevo DF `kivaRawNoNullDF` en el que se hayan eliminado todas las filas que tengan algún valor nulo en cualquier columna **excepto en la columna tags**, que no será relevante para el análisis y por tanto podemos ignorar sus valores nulos y mantener dichas filas.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR LAS VARIABLES
# YOUR CODE HERE
kivaRawDF = spark.read.option("header", "true")\
                      .option("quote", "\"")\
                      .option("escape", "\"")\
                      .option("mode", "DROPMALFORMED")\
                      .csv("abfss://datos@masterncv001sta.dfs.core.windows.net/kiva_loans.csv")
columnasExceptoTags = kivaRawDF.columns
columnasExceptoTags.remove("tags")
kivaRawNoNullDF = kivaRawDF.na.drop(subset = columnasExceptoTags) #Elimnamos los valore nulos excepto en la columna tags. 



In [0]:
from pyspark.sql.types import DoubleType
assert(kivaRawNoNullDF.count() == 574115)

**(1 punto)** Ejercicio 2

* Las columnas `posted_time`, `disbursed_time` y `funded_time` son en realidad instantes de tiempo que Spark debería procesar como timestamp. Partiendo de `kivaRawNoNullDF`, reemplaza **ambas columnas** por su versión convertida a timestamp, utilizando `withColumn` con el mismo nombre de cada columna, y donde el nuevo valor de la columna viene dado por el siguiente código:

        F.from_unixtime(F.unix_timestamp('nombreColumna', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))

* Además, convierte a `DoubleType` la columna `loan_amount` y a `IntegerType` la columna `term_in_months`.

* El DF resultante de todas estas operaciones debe quedar almacenado en la variable `kivaDF`, **cacheado**.


In [0]:
# No olvides los imports que necesites...
from pyspark.sql import types as T
from pyspark.sql import functions as F
# LÍNEAS EVALUABLES, NO RENOMBRAR LAS VARIABLES
# YOUR CODE HERE
kivaDF = kivaRawNoNullDF.withColumn("posted_time",
                                     F.from_unixtime(F.unix_timestamp('posted_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))\
                                         .withColumn("disbursed_time", F.from_unixtime(F.unix_timestamp('disbursed_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))\
                                         .withColumn("funded_time", F.from_unixtime(F.unix_timestamp('funded_time', 'yyyy-MM-dd HH:mm:ssXXX')).cast("timestamp"))\
                                         .withColumn("loan_amount", F.col("loan_amount").cast(T.DoubleType()))\
                                         .withColumn("term_in_months",F.col("term_in_months").cast(T.IntegerType()))\
                                         .cache()


In [0]:
from pyspark.sql import functions as F
typesDict = dict(kivaDF.dtypes)
assert(typesDict["posted_time"] == "timestamp") 
assert(typesDict["disbursed_time"] == "timestamp") 
assert(typesDict["loan_amount"] == "double") 
assert(typesDict["term_in_months"] == "int")
cnt_cond = lambda cond: F.sum(F.when(cond, 1).otherwise(0))
nullsRow = kivaDF.select(cnt_cond(F.col("posted_time").isNull()).alias("posted_nulls"),
              cnt_cond(F.col("disbursed_time").isNull()).alias("disbursed_nulls")).head()
assert(nullsRow.posted_nulls == 0)
assert(nullsRow.disbursed_nulls == 0)

**(2 puntos)** Ejercicio 3

Partiendo de `kivaDF`:

* Primero, añade una nueva columna `dias_desembolso` que contenga el número de días que han pasado entre la fecha en que los prestamistas aceptaron financiar un proyecto, y la fecha en que el agente de campo entregó los fondos al beneficiario. Para ello, utiliza `withColumn` en combinación con la función `F.datediff("colFuturo", "colPasado")`
* De manera análoga, añade otra nueva columna `dias_aceptacion` que contenga el número de días entre el anuncio de la necesidad de préstamo y la aceptación de financiarlo por parte de algún prestamista.
* Reemplazar la columna `sector` por otra en la que se hayan traducido las categorías "Education" por "Educacion" (sin tilde para evitar posibles problemas) y "Agriculture" por "Agricultura", dejando como están el resto de categorías. **La sustitución no debe tener más que tres casos**: uno para cada categoría que vamos a reemplazar, y un tercero para el resto de categorías, que deben quedarse como estaban.
* El resultado debe quedar guardado en la variable `kivaTiemposDF`.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
# imports......
from pyspark.sql import functions as F
# YOUR CODE HERE

kivaTiemposDF = kivaDF.withColumn("dias_desembolso",
                                  F.datediff("disbursed_time", "funded_time"))\
                                      .withColumn("dias_aceptacion",F.datediff("funded_time", "posted_time"))\
                                      .withColumn("sector",
                                                   F.when(F.col("sector") == "Agriculture","Agricultura")\
                                                    .when(F.col("sector") == "Education","Educacion")\
                                                    .otherwise(F.col("sector"))) 


In [0]:
assert(kivaTiemposDF.where("sector == 'Agricultura'").count() == 157003)
assert(kivaTiemposDF.where("sector == 'Educacion'").count() == 28417)
# Comprobamos que las 13 restantes se mantienen sin cambios
assert(kivaTiemposDF.groupBy("sector").count().join(kivaDF.groupBy("sector").count(), ["sector", "count"]).count() == 13)

**(3 puntos)** Ejercicio 4

Partiendo de `kivaTiemposDF`, crear un nuevo DataFrame llamado `kivaAgrupadoDF` que tenga:

* Tantas filas como **países (`country`; no usar el código de país)**, y tantas columnas como **sectores** (cada una llamada como el sector) más una (la columna del país, que debe aparecer en primer lugar). En cada celda deberá ir el número **medio (redondeado a 2 cifras decimales)** de días transcurridos en ese país y sector *entre la fecha en que se anuncia la necesidad de préstamo y la fecha en la que un prestamista acepta financiarlo*. Esta columna ha sido calculada en la celda precedente.
* Después de esto, añadir una columna adicional `transcurrido_global` con el número **medio (redondeado a 2 cifras decimales) de días transcurridos en cada país** entre ambas fechas *sin tener en cuenta el sector*. Cada fila tendrá la media de las 15 columnas del apartado anterior.
* Por último, ordenar el DF resultante **descendentemente** en base al tiempo medio global, `transcurrido_global`. El DF resultado de la ordenación debe ser almacenado en la variable `kivaAgrupadoDF`. 

PISTA: utiliza el método `pivot` con el sector para el primer apartado, envolviendo a la función de agregación con la función `F.round`, es decir, `F.round(F.funcionAgregacion(....), 2)`, y `withColumn` con una operación aritmética entre columnas en el segundo. **No debe utilizarse la función `when`** ya que Spark es capaz de hacer directamente aritmética entre objetos columna. La operación aritmética también debe estar envuelta por round: `F.round(op. aritmética entre objetos columna, 2)`.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
# YOUR CODE HERE

kivaAgrupadoDF = kivaTiemposDF.groupBy("country")\
                              .pivot("sector")\
                              .agg(F.round(F.mean(F.col("dias_aceptacion")), 2))\
                              .withColumn("transcurrido_global",
                                          F.round((sum([F.col(sector) for sector in ["Agricultura", "Arts", 
                                                                                     "Clothing",
                                                                                     "Construction","Educacion","Entertainment","Food","Health","Housing", "Manufacturing","Personal Use","Retail","Services","Transportation","Wholesale"]])/15),2))\
                             .sort(F.desc("transcurrido_global"))                            

In [0]:
r1 = kivaAgrupadoDF.head()
assert(r1.country == "United States")
assert((r1.Agricultura - 12.0 < 0.01) | (r1.Agricultura - 12.17 < 0.01))
assert((r1.Educacion - 15.21 < 0.01) | (r1.Educacion - 15.33 < 0.01))
assert(r1.Wholesale - 27.5 < 0.01)
assert((r1.transcurrido_global - 20.94 < 0.01) | (r1.transcurrido_global - 21.04 < 0.01))

**(3 puntos)** Ejercicio 5

Partiendo de nuevo de `kivaTiemposDF`, añadir las siguientes columnas:

* Primero, tres columnas adicionales llamadas `transc_medio`, `transc_min`, `transc_max` que contengan, respectivamente, **el número de días medio, mínimo y máximo transcurrido para proyectos de ese mismo país y ese mismo sector** entre la fecha en que se postea la necesidad de préstamo y la fecha en la que alguien acepta financiarlo (es decir, la columna `dias_aceptacion` calculada antes y utilizada también en la celda anterior). Es decir, queremos una columna extra para que podamos tener, junto a cada préstamo, información agregada de los préstamos similares, entendidos como aquellos del mismo país y del mismo sector. **No se debe utilizar JOIN sino solo funciones de ventana**.
* Finalmente, crear otra columna adicional `diff_dias` que contenga la **diferencia en días entre los días que transcurrieron en este proyecto y la media de días de los proyectos similares** (calculada en el apartado anterior). Debería ser lo primero menos lo segundo, de forma que un número positivo indica que este préstamo tardó más días en ser aceptado que la media de días de este país y sector, y un número negativo indica lo contrario. El resultado debe obtenerse aplicando operaciones aritméticas con columnas existentes, **sin utilizar `when`**.
* El DF resultante con las 4 columnas nuevas que hemos añadido debe quedar almacenado en la variable `kivaExtraInfoDF`.

In [0]:
# LÍNEA EVALUABLE, NO RENOMBRAR VARIABLES
# imports necesarios
from pyspark.sql import Window
# YOUR CODE HERE
windowPaisSector = Window().partitionBy("country","sector")
kivaExtraInfoDF = kivaTiemposDF.withColumn("transc_medio",
                                            F.mean("dias_aceptacion").over(windowPaisSector))\
                               .withColumn("transc_min",
                                           F.min("dias_aceptacion").over(windowPaisSector))\
                                .withColumn("transc_max",
                                            F.max("dias_aceptacion").over(windowPaisSector))\
                                .withColumn("diff_dias",
                                            F.col("dias_aceptacion")-F.col("transc_medio") )
kivaExtraInfoDF.limit(10).display()



id,funded_amount,loan_amount,activity,sector,use,country_code,country,region,currency,partner_id,posted_time,disbursed_time,funded_time,term_in_months,lender_count,tags,borrower_genders,repayment_interval,date,dias_desembolso,dias_aceptacion,transc_medio,transc_min,transc_max,diff_dias
805613,6000.0,6000.0,Textiles,Arts,To buy 9 sewing machines that will increase productivity and to provide safer transportation to the female workers.,AF,Afghanistan,Kandahar City,USD,412.0,2014-11-26T00:51:29Z,2015-01-07T08:00:00Z,2014-11-26T20:07:08Z,8,204,"user_favorite, user_favorite, #Biz Durable Asset, #Woman Owned Biz, user_favorite, user_favorite, user_favorite, #Fabrics, #Hidden Gem, #Interesting Photo, #Inspiring Story, #Post-disbursed, user_favorite, user_favorite",female,bullet,2014-11-26,42,0,0.5,0,1,-0.5
1023542,8000.0,8000.0,Textiles,Arts,"To purchase raw materials, including fabrics, threads, and other embroidery supplies, to finish preparing for the Santa Fe Folk Art Market in July 2016",AF,Afghanistan,Kandahar City,USD,412.0,2016-02-16T15:15:51Z,2016-04-05T07:00:00Z,2016-02-17T05:25:42Z,6,308,"#Fabrics, user_favorite, user_favorite, #Unique, #Woman Owned Biz, #Repeat Borrower, #Inspiring Story, user_favorite, user_favorite, user_favorite, user_favorite, user_favorite, #Job Creator, user_favorite, user_favorite, user_favorite, user_favorite, user_favorite, user_favorite, user_favorite",female,bullet,2016-02-16,48,1,0.5,0,1,0.5
662240,1475.0,1475.0,Cafe,Food,to buy equipment for her bar-cafe.,AL,Albania,Elbasan,ALL,239.0,2014-01-27T10:26:03Z,2014-01-24T08:00:00Z,2014-01-29T17:01:30Z,26,55,"volunteer_like, #Health and Sanitation, #Supporting Family, #Woman Owned Biz",female,monthly,2014-01-27,-5,2,21.865384615384617,2,44,-19.865384615384617
666820,2900.0,2900.0,Bakery,Food,to build an additional space in her bakery to sell groceries,AL,Albania,Korce,ALL,239.0,2014-02-07T11:32:56Z,2014-02-05T08:00:00Z,2014-02-17T17:57:30Z,26,112,"user_favorite, user_favorite",female,monthly,2014-02-07,-12,10,21.865384615384617,2,44,-11.865384615384617
674498,1975.0,1975.0,Grocery Store,Food,to add to her grocery store a refrigerator and counters,AL,Albania,Korce,ALL,239.0,2014-02-21T11:32:34Z,2014-02-17T08:00:00Z,2014-02-26T19:21:25Z,32,74,null,female,monthly,2014-02-21,-9,5,21.865384615384617,2,44,-16.865384615384617
702586,1000.0,1000.0,Food Market,Food,to buy kitchen supplies and ware.,AL,Albania,Lezhe,ALL,239.0,2014-04-25T11:17:25Z,2014-04-24T07:00:00Z,2014-05-17T22:32:13Z,20,36,"volunteer_pick, volunteer_like, user_favorite, user_favorite",female,monthly,2014-04-25,-23,22,21.865384615384617,2,44,0.13461538461538325
706185,1500.0,1500.0,Grocery Store,Food,to buy shelves and inventory for her store.,AL,Albania,Korce,ALL,239.0,2014-05-06T10:41:36Z,2014-04-30T07:00:00Z,2014-05-15T23:45:17Z,21,47,"#Interesting Photo, #Tourism, #Woman Owned Biz",female,monthly,2014-05-06,-15,9,21.865384615384617,2,44,-12.865384615384617
709239,1000.0,1000.0,Grocery Store,Food,to buy inventory for her grocery store.,AL,Albania,Korce,ALL,239.0,2014-05-13T08:34:54Z,2014-05-08T07:00:00Z,2014-05-19T19:17:41Z,26,36,null,female,monthly,2014-05-13,-11,6,21.865384615384617,2,44,-15.865384615384617
715980,2450.0,2450.0,Grocery Store,Food,to finish the grocery store he has started building,AL,Albania,Korce,ALL,239.0,2014-05-26T09:55:30Z,2014-05-20T07:00:00Z,2014-06-28T17:55:15Z,20,92,"#Parent, #Biz Durable Asset, user_favorite",male,monthly,2014-05-26,-39,33,21.865384615384617,2,44,11.134615384615383
719122,1475.0,1475.0,Grocery Store,Food,"who was born deaf and mute, to stock up on inventory items for her little village grocery store.",AL,Albania,Korce,ALL,239.0,2014-06-02T09:38:17Z,2014-05-29T07:00:00Z,2014-06-10T14:06:36Z,20,52,"user_favorite, #Parent, #Widowed, #Woman Owned Biz, user_favorite",female,monthly,2014-06-02,-12,8,21.865384615384617,2,44,-13.865384615384617


In [0]:
r = kivaExtraInfoDF.where("id = '658540'").head()
assert(r.country == 'Burkina Faso')
assert(r.transc_medio - 11.02 < 0.05)
assert(r.dias_aceptacion == 35)
assert(r.diff_dias - 24.0 < 0.001)